# Braket via Nexus

Here you can find some examples and explanations on running quantum circuits on [AWS Braket](https://aws.amazon.com/braket/) via Nexus. 

These will require that you have set Braket credentials via Settings->Linked Accounts on the Nexus website.

In [ ]:
from datetime import datetime

from pytket import Circuit

import qnexus as qnx

List available Braket devices, optionally specifying a specific AWS region (by default devices for ALL regions will be returned).

In [ ]:
qnx.devices.get_all(issuers=[qnx.devices.IssuerEnum.BRAKET]).df()

To configure Nexus to target Braket, you'll need to provide the following arguments to the BraketConfig.

- `local`: `bool`
    - if true then the circuit will run as a simulation in the Nexus cloud.
- `device_type`: `str`
    - device type from device ARN (e.g. `"qpu"`, `"quantum-simulator"`)
- `provider`: `str`
    - provider name from device ARN (e.g. `"ionq"`, `"rigetti"`, `"oqc"`, `"amazon"`, ...)
- `device`: `str`
    - device name from device ARN (e.g. `"ionQdevice"`, `"Aspen-8"`, `"sv1"`, ...)
- `s3_bucket`: `str`
    - name of S3 bucket to store results
- `s3_folder`: `str`
    - name of folder (`"key"`) in S3 bucket to store results in

In [ ]:
braket_config = qnx.BraketConfig(
    local=True,
    device_type="quantum-simulator",
    provider="amazon",
    device="sv1",
    s3_bucket="",
    s3_folder="",
)

## Example of compiling and executing a circuit on AWS Braket

In [ ]:
my_project_ref = qnx.projects.get_or_create(name="My AWS Braket Project")

circuit = Circuit(2).H(0).CX(0, 1).measure_all()

my_circuit_ref = qnx.circuits.upload(
    name=f"My AWS Braket Circuit from {datetime.now()}",
    circuit=circuit,
    project=my_project_ref,
)

compile_job = qnx.start_compile_job(
    circuits=[my_circuit_ref],
    name=f"My Compile Job from {datetime.now()}",
    optimisation_level=1,
    backend_config=braket_config,
    project=my_project_ref,
)

qnx.jobs.wait_for(compile_job)

compiled_circuits = [item.get_output() for item in qnx.jobs.results(compile_job)]

execute_job_ref = qnx.start_execute_job(
    circuits=compiled_circuits,
    name=f"My Execute Job from {datetime.now()}",
    n_shots=[100],
    backend_config=braket_config,
    project=my_project_ref,
)

qnx.jobs.wait_for(execute_job_ref)


# Retrieve a ExecutionResultRef for every Circuit that was executed
execute_job_result_refs = qnx.jobs.results(execute_job_ref)

# Get a pytket BackendResult for the execution
result = execute_job_result_refs[0].download_result()

result.get_counts()